In [84]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- join death rate, sales, production, policy, and tobacco usage (worldwide)

In [2]:
with open('config/EDA.json') as fh:
    configs = json.load(fh)
    DR_PATH, SALES_PATH, PRODUCTION_PATH, EDA_VIZ_PATH,\
    POLICY_PATH, TBC_WW_PATH, = configs['DR_PATH'], \
    configs['SALES_PATH'], configs['PRODUCTION_PATH'], configs['EDA_VIZ_PATH'], configs['POLICY_PATH'], configs['TBC_WW_PATH']

In [74]:
death_rate

,Entity,Code,Year,All_ages,Under_5,5_14,50_69,15_49,70_plus,imputed_code
0,Afghanistan,AFG,1990,63.895905,NaN,NaN,267.230009,16.589519,679.006755,AFG
1,Afghanistan,AFG,1991,61.846347,NaN,NaN,266.975516,15.456913,677.617648,AFG
2,Afghanistan,AFG,1992,53.436511,NaN,NaN,266.430053,12.767999,679.505810,AFG
3,Afghanistan,AFG,1993,47.044347,NaN,NaN,267.969428,11.000425,683.973588,AFG
4,Afghanistan,AFG,1994,45.799808,NaN,NaN,272.403687,10.738020,691.007773,AFG
...,...,...,...,...,...,...,...,...,...,...
6463,Zimbabwe,ZWE,2013,66.609741,NaN,NaN,430.024361,24.046022,1275.647333,ZWE
6464,Zimbabwe,ZWE,2014,63.983527,NaN,NaN,408.855801,22.964390,1220.939407,ZWE
6465,Zimbabwe,ZWE,2015,62.211166,NaN,NaN,392.380248,22.393477,1174.938436,ZWE
6466,Zimbabwe,ZWE,2016,60.644339,NaN,NaN,378.003366,21.768089,1124.960820,ZWE


In [3]:
death_rate, sales, policy, production, tbc_ww = pd.read_csv(DR_PATH), pd.read_csv(SALES_PATH),\
    pd.read_csv(POLICY_PATH), pd.read_csv(PRODUCTION_PATH), pd.read_csv(TBC_WW_PATH)

In [4]:
display(death_rate.head(2))
display(sales.head(2))
display(policy.head(2))
display(production.head(2))
display(tbc_ww.head(2))

,Entity,Code,Year,All_ages,Under_5,5_14,50_69,15_49,70_plus
0,Afghanistan,AFG,1990,63.895905,NaN,NaN,267.230009,16.589519,679.006755
1,Afghanistan,AFG,1991,61.846347,NaN,NaN,266.975516,15.456913,677.617648


,Entity,Code,Year,amt
0,Armenia,ARM,1988,12.0
1,Armenia,ARM,1989,11.6


,Entity,Code,Year,AvgCigarettePriceDollars,AvgTaxesAsPctCigarettePrice,EnforceBansTobaccoAd,HelpToQuit
0,Algeria,DZA,2012,1.84,47.0,4,3
1,Algeria,DZA,2014,2.09,40.9,4,4


,Country or Area,Year,Unit,Value,Value Footnotes
0,Albania,2006,Metric tons,546.600000,NaN
1,Albania,2006,Mil. USD,1.324113,NaN


,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Year,Gender,Value
0,SEAR,South-East Asia,PRK,Democratic People's Republic of Korea,2018,Female,0.0
1,EUR,Europe,AZE,Azerbaijan,2018,Female,0.2


In [53]:
np.isnan(death_rate.loc[death_rate['Code'].isna()]['Code'].iloc[0])

True

In [54]:
def impute_code(row):
#     print(row['Code'])
    if pd.isna(row['Code']):
        return row['Entity']
    else:
        return row['Code']

In [55]:
death_rate['imputed_code'] = death_rate.apply(impute_code, axis = 1)
sales['imputed_code'] = sales.apply(impute_code,axis = 1)
policy['imputed_code'] = policy.apply(impute_code,axis=1)

In [60]:
death_and_sales = (death_rate.merge(sales, how='inner', on = ['imputed_code','Year']))\
    .drop(['Entity_y', 'Code_y'], axis=1).rename({'Entity_x': 'Entity', 'Code_x': 'Code'}, axis=1)
death_and_sales.head()

,Entity,Code,Year,All_ages,Under_5,5_14,50_69,15_49,70_plus,imputed_code,amt
0,Armenia,ARM,1990,121.362447,NaN,NaN,449.915189,21.899873,1131.267472,ARM,11.9
1,Armenia,ARM,1991,131.006209,NaN,NaN,488.825199,23.553359,1189.114059,ARM,9.3
2,Armenia,ARM,1992,139.504601,NaN,NaN,523.378387,26.587575,1229.115731,ARM,4.9
3,Armenia,ARM,1993,144.812587,NaN,NaN,552.176897,29.055858,1241.817610,ARM,5.5
4,Armenia,ARM,1994,143.491999,NaN,NaN,551.009746,29.990658,1206.602223,ARM,5.7


In [61]:
# set(death_rate['imputed_code']) - set(sales['imputed_code'])

join production

In [62]:
set(death_and_sales['Entity']) - (set(production['Country or Area']))

{'Armenia',
 'Belarus',
 'Czechia',
 'Georgia',
 'Israel',
 'Moldova',
 'Russia',
 'Spain',
 'United States'}

In [63]:
country_replacement = {'United States of America': 'United States'}
production['imputed_entity'] = production['Country or Area'].replace(country_replacement)

In [64]:
production_mt = production.copy().loc[production['Unit']=='Metric tons'].drop('Unit', axis=1).rename({'Value':'metric_tons'},axis=1)
production_usd = production.copy().loc[production['Unit'] == 'Mil. USD'].drop('Unit', axis=1).rename({'Value':'mil_usd'},axis=1)

In [67]:
death_sales_production = death_and_sales\
    .merge(production_mt, how='left', left_on=['Entity','Year'], right_on=['imputed_entity','Year'])\
    .merge(production_usd, how='left', left_on=['Entity','Year'], right_on=['imputed_entity','Year'])\
    .rename({'Value Footnotes_x':'tons_footnote', 'Value Footnotes_y':'usd_footnote'},axis=1)\
    .drop(['Country or Area_x',
       'imputed_entity_x','Country or Area_y','imputed_entity_y'],axis=1)
death_sales_production.shape

(840, 15)

In [68]:
death_sales_production_policies = death_sales_production\
    .merge(policy, how='left', on=['imputed_code', 'Year']).drop(['Entity_y', 'Code_y'], axis=1)\
    .rename({'Entity_x': 'Entity', 'Code_x': 'Code'}, axis=1)
death_sales_policies.shape

(840, 15)

In [71]:
death_sales_production_policies.to_csv('data/ww_synthesized.csv')

In [72]:
death_sales_production_policies

,Entity,Code,Year,All_ages,Under_5,5_14,50_69,15_49,70_plus,imputed_code,amt,metric_tons,tons_footnote,mil_usd,usd_footnote,AvgCigarettePriceDollars,AvgTaxesAsPctCigarettePrice,EnforceBansTobaccoAd,HelpToQuit
0,Armenia,ARM,1990,121.362447,NaN,NaN,449.915189,21.899873,1131.267472,ARM,11.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Armenia,ARM,1991,131.006209,NaN,NaN,488.825199,23.553359,1189.114059,ARM,9.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Armenia,ARM,1992,139.504601,NaN,NaN,523.378387,26.587575,1229.115731,ARM,4.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Armenia,ARM,1993,144.812587,NaN,NaN,552.176897,29.055858,1241.817610,ARM,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Armenia,ARM,1994,143.491999,NaN,NaN,551.009746,29.990658,1206.602223,ARM,5.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,United States,USA,2010,136.471977,0.0,0.0,212.449950,10.972701,926.944926,USA,3.6,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5.0
836,United States,USA,2011,135.564825,0.0,0.0,207.953219,10.360038,912.371554,USA,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
837,United States,USA,2012,133.393056,0.0,0.0,203.137541,9.643943,884.769941,USA,3.5,NaN,NaN,NaN,NaN,6.07,42.9,2.0,5.0
838,United States,USA,2013,131.994841,0.0,0.0,198.989774,9.057056,858.231481,USA,3.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
tbc_ww.head()

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Year,Gender,Value
0,SEAR,South-East Asia,PRK,Democratic People's Republic of Korea,2018,Female,0.0
1,EUR,Europe,AZE,Azerbaijan,2018,Female,0.2
2,AFR,Africa,ERI,Eritrea,2018,Female,0.3
3,EMR,Eastern Mediterranean,EGY,Egypt,2018,Female,0.4
4,AFR,Africa,GHA,Ghana,2018,Female,0.4


In [79]:
death_use_sales_production_policies = death_sales_production_policies\
    .merge(tbc_ww, how='inner', left_on=['Year', 'Code'], right_on=['Year', 'SpatialDimValueCode'])

In [83]:
death_use_sales_production_policies.to_csv('data/ww_small_subset.csv')

sales vs usage?